# RAG 베이스라인 파이프라인

Retrieval-Augmented Generation 전체 파이프라인을 구현하고 테스트합니다.

In [ ]:
import sys
from pathlib import Path
import os
from getpass import getpass
import time
from datetime import datetime

# 프로젝트 루트 경로
project_root = Path().absolute().parent
sys.path.append(str(project_root))

from src import config
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage

print("RAG 베이스라인 파이프라인")
print("="*60)

## 1. API Key 및 ChromaDB Manager 로드

In [ ]:
# API Key 설정
api_key = config.OPENAI_API_KEY

if not api_key:
    print("OpenAI API Key를 입력하세요:")
    api_key = getpass("API Key: ")
    os.environ['OPENAI_API_KEY'] = api_key

print("✅ API Key 설정 완료")

In [ ]:
# ChromaDB Manager 로드
try:
    %store -r chroma_manager
    print("✅ ChromaDB Manager 로드 완료")
except:
    print("⚠️ 저장된 ChromaDB Manager가 없습니다. 새로 생성합니다.")
    from src.vectorstore import ChromaDBManager
    chroma_manager = ChromaDBManager(api_key=api_key)
    chroma_manager.load_vectorstore()

# 컬렉션 정보 확인
info = chroma_manager.get_collection_info()
print(f"\n컬렉션: {info.get('name', 'N/A')}")
print(f"문서 수: {info.get('count', 0):,}개")

## 2. LLM 초기화

In [ ]:
# LLM 설정
llm = ChatOpenAI(
    model=config.LLM_MODEL,
    temperature=config.TEMPERATURE,
    max_tokens=config.MAX_TOKENS,
    openai_api_key=api_key
)

print(f"LLM 모델: {config.LLM_MODEL}")
print(f"Temperature: {config.TEMPERATURE}")
print(f"Max Tokens: {config.MAX_TOKENS}")
print("\n✅ LLM 초기화 완료")

## 3. RAG 프롬프트 템플릿

In [ ]:
# 시스템 프롬프트
system_prompt = """당신은 RFP(제안요청서) 분석 전문가입니다.
주어진 문서 내용을 바탕으로 사용자의 질문에 정확하고 상세하게 답변해주세요.

답변 작성 시 다음 규칙을 따르세요:
1. 반드시 제공된 문서의 내용만을 사용하여 답변하세요.
2. 문서에 없는 내용은 추측하지 말고 "문서에 해당 정보가 없습니다"라고 답변하세요.
3. 답변은 명확하고 구체적으로 작성하세요.
4. 가능한 경우 문서의 어느 부분에서 정보를 얻었는지 언급하세요.
5. 한국어로 답변하세요.
"""

# 프롬프트 템플릿
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", """참고 문서:
{context}

질문: {question}

답변:""")
])

print("✅ 프롬프트 템플릿 생성 완료")

## 4. RAG 파이프라인 함수

In [ ]:
def rag_query(
    question: str,
    top_k: int = config.TOP_K,
    filter: dict = None,
    verbose: bool = True
) -> dict:
    """
    RAG 파이프라인 실행
    
    Args:
        question: 사용자 질문
        top_k: 검색할 문서 수
        filter: 메타데이터 필터
        verbose: 상세 출력 여부
    
    Returns:
        결과 딕셔너리 (answer, sources, retrieved_docs, response_time)
    """
    start_time = time.time()
    
    # 1. Retrieval
    if verbose:
        print("[1/3] 문서 검색 중...")
    
    retrieved_docs = chroma_manager.similarity_search_with_score(
        query=question,
        k=top_k,
        filter=filter
    )
    
    if not retrieved_docs:
        return {
            'answer': "관련 문서를 찾을 수 없습니다.",
            'sources': [],
            'retrieved_docs': [],
            'response_time': time.time() - start_time
        }
    
    if verbose:
        print(f"   → {len(retrieved_docs)}개 문서 검색 완료")
    
    # 2. Context 구성
    if verbose:
        print("[2/3] 컨텍스트 구성 중...")
    
    context = "\n\n".join([
        f"[문서 {i+1}] (출처: {doc.metadata.get('file_name', 'Unknown')})\n{doc.page_content}"
        for i, (doc, score) in enumerate(retrieved_docs)
    ])
    
    # 3. Generation
    if verbose:
        print("[3/3] 답변 생성 중...")
    
    messages = prompt_template.format_messages(
        context=context,
        question=question
    )
    
    response = llm.invoke(messages)
    answer = response.content
    
    # 출처 정보
    sources = list(set([
        doc.metadata.get('file_name', 'Unknown')
        for doc, score in retrieved_docs
    ]))
    
    response_time = time.time() - start_time
    
    if verbose:
        print(f"\n✅ 완료 (소요 시간: {response_time:.2f}초)\n")
    
    return {
        'answer': answer,
        'sources': sources,
        'retrieved_docs': retrieved_docs,
        'response_time': response_time
    }

print("✅ RAG 파이프라인 함수 정의 완료")

## 5. 단일 질문 테스트

In [ ]:
# 테스트 질문
test_question = "국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘."

print(f"질문: {test_question}\n")
print("="*60)

result = rag_query(test_question, verbose=True)

print("="*60)
print(f"\n답변:\n{result['answer']}")
print(f"\n출처: {', '.join(result['sources'])}")
print(f"소요 시간: {result['response_time']:.2f}초")

## 6. 프로젝트 가이드의 예시 질문 테스트

In [ ]:
# 프로젝트 가이드의 질문 예시
guide_questions = [
    "국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘.",
    "콘텐츠 개발 관리 요구 사항에 대해서 더 자세히 알려 줘.",
    "교육이나 학습 관련해서 다른 기관이 발주한 사업은 없나?",
    "기초과학연구원 극저온시스템 사업 요구에서 AI 기반 예측에 대한 요구사항이 있나?",
    "한국 원자력 연구원에서 선량 평가 시스템 고도화 사업을 발주했는데, 이 사업이 왜 추진되는지 목적을 알려 줘."
]

print("프로젝트 가이드 예시 질문 테스트\n")
print("="*60)

for i, question in enumerate(guide_questions, 1):
    print(f"\n[질문 {i}] {question}\n")
    
    result = rag_query(question, verbose=False)
    
    print(f"답변: {result['answer'][:300]}...")
    print(f"출처: {', '.join(result['sources'][:2])}")
    print(f"시간: {result['response_time']:.2f}초")
    print("-"*60)

## 7. 대화형 인터페이스

In [ ]:
def interactive_rag():
    """
    대화형 RAG 인터페이스
    'quit' 또는 'exit' 입력 시 종료
    """
    print("\n" + "="*60)
    print("대화형 RAG 시스템 (종료: 'quit' 또는 'exit')")
    print("="*60 + "\n")
    
    while True:
        question = input("\n질문: ")
        
        if question.lower() in ['quit', 'exit', '종료']:
            print("\n종료합니다.")
            break
        
        if not question.strip():
            continue
        
        print("\n" + "-"*60)
        result = rag_query(question, verbose=False)
        print("-"*60)
        
        print(f"\n답변:\n{result['answer']}")
        print(f"\n출처: {', '.join(result['sources'])}")
        print(f"소요 시간: {result['response_time']:.2f}초")

# 실행하려면 주석 해제
# interactive_rag()

## 8. 성능 평가 준비

In [ ]:
# 평가용 질문 세트
evaluation_questions = [
    "국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘.",
    "콘텐츠 개발 관리 요구 사항에 대해서 더 자세히 알려 줘.",
    "교육이나 학습 관련해서 다른 기관이 발주한 사업은 없나?",
    "기초과학연구원 극저온시스템 사업 요구에서 AI 기반 예측에 대한 요구사항이 있나?",
    "그럼 모니터링 업무에 대한 요청사항이 있는지 찾아보고 알려 줘.",
    "한국 원자력 연구원에서 선량 평가 시스템 고도화 사업을 발주했는데, 이 사업이 왜 추진되는지 목적을 알려 줘.",
    "고려대학교 차세대 포털 시스템 사업이랑 광주과학기술원의 학사 시스템 기능개선 사업을 비교해 줄래?",
    "고려대학교랑 광주과학기술원 각각 응답 시간에 대한 요구사항이 있나? 문서를 기반으로 정확하게 답변해 줘.",
]

print(f"평가용 질문 세트: {len(evaluation_questions)}개")

# 평가 결과 저장
evaluation_results = []

for i, question in enumerate(evaluation_questions, 1):
    print(f"\n[{i}/{len(evaluation_questions)}] 평가 중...")
    result = rag_query(question, verbose=False)
    
    evaluation_results.append({
        'question': question,
        'answer': result['answer'],
        'sources': result['sources'],
        'response_time': result['response_time'],
        'num_sources': len(result['sources'])
    })
    
    print(f"   완료 ({result['response_time']:.2f}초)")

print("\n✅ 평가 완료")

In [ ]:
# 평가 결과 요약
import numpy as np

response_times = [r['response_time'] for r in evaluation_results]
num_sources_list = [r['num_sources'] for r in evaluation_results]

print("\n베이스라인 성능 요약")
print("="*60)
print(f"\n평가 질문 수: {len(evaluation_results)}개")
print(f"\n응답 시간:")
print(f"  평균: {np.mean(response_times):.2f}초")
print(f"  중앙값: {np.median(response_times):.2f}초")
print(f"  최소: {np.min(response_times):.2f}초")
print(f"  최대: {np.max(response_times):.2f}초")
print(f"\n출처 문서 수:")
print(f"  평균: {np.mean(num_sources_list):.1f}개")
print(f"  최소: {np.min(num_sources_list)}개")
print(f"  최대: {np.max(num_sources_list)}개")

In [ ]:
# 평가 결과 상세 출력
print("\n평가 결과 상세:\n")

for i, result in enumerate(evaluation_results, 1):
    print(f"[질문 {i}]")
    print(f"Q: {result['question']}")
    print(f"A: {result['answer'][:200]}...")
    print(f"출처: {', '.join(result['sources'])}")
    print(f"시간: {result['response_time']:.2f}초")
    print("-"*60 + "\n")

## 9. 결과 저장

In [ ]:
import json

# 결과를 JSON으로 저장
results_dir = project_root / "data" / "processed"
results_dir.mkdir(parents=True, exist_ok=True)

results_file = results_dir / f"baseline_evaluation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

with open(results_file, 'w', encoding='utf-8') as f:
    json.dump({
        'timestamp': datetime.now().isoformat(),
        'config': {
            'chunk_size': config.CHUNK_SIZE,
            'chunk_overlap': config.CHUNK_OVERLAP,
            'embedding_model': config.EMBEDDING_MODEL,
            'llm_model': config.LLM_MODEL,
            'temperature': config.TEMPERATURE,
            'top_k': config.TOP_K
        },
        'results': evaluation_results,
        'summary': {
            'num_questions': len(evaluation_results),
            'avg_response_time': float(np.mean(response_times)),
            'avg_num_sources': float(np.mean(num_sources_list))
        }
    }, f, ensure_ascii=False, indent=2)

print(f"✅ 평가 결과 저장 완료: {results_file}")

## 10. 베이스라인 완성 체크리스트

In [ ]:
print("\n" + "="*60)
print("베이스라인 RAG 시스템 완성 체크리스트")
print("="*60 + "\n")

checklist = [
    "✅ 문서 로딩 (PDF + HWP)",
    "✅ 문서 청킹",
    "✅ 임베딩 생성",
    "✅ ChromaDB Vector Store 구축",
    "✅ Retrieval 기능 구현",
    "✅ LLM 답변 생성",
    "✅ RAG 파이프라인 통합",
    "✅ 성능 평가",
    "✅ 결과 저장"
]

for item in checklist:
    print(item)

print("\n" + "="*60)
print("베이스라인 구축 완료!")
print("="*60)

## 다음 단계

### 개선 방향

1. **Retrieval 고도화**
   - Multi-Query Retrieval
   - Re-Ranking
   - Hybrid Search (BM25 + Vector)
   - 메타데이터 필터링 활용

2. **Generation 개선**
   - 프롬프트 엔지니어링
   - Few-shot Learning
   - Chain-of-Thought
   - 답변 포맷 구조화

3. **대화 히스토리**
   - 이전 대화 맥락 유지
   - 후속 질문 처리

4. **평가 체계**
   - Retrieval 평가 (Precision, Recall)
   - Generation 평가 (Faithfulness, Relevance)
   - End-to-End 평가

5. **사용자 인터페이스**
   - Streamlit 웹 앱 (선택)